In [1]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import RFE, RFECV,SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, f1_score

### Carregar dados e visualizar -  Regressao

In [2]:
# Carregar dados
df = pd.read_csv('./datasets/colesterol.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  1000 non-null   int64  
 1   Grupo Sanguíneo     1000 non-null   object 
 2   Fumante             1000 non-null   object 
 3   Nível de Atividade  1000 non-null   object 
 4   Idade               1000 non-null   int64  
 5   Peso                1000 non-null   float64
 6   Altura              1000 non-null   int64  
 7   Colesterol          1000 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 62.6+ KB


In [4]:
df.head(10)

,Id,Grupo Sanguíneo,Fumante,Nível de Atividade,Idade,Peso,Altura,Colesterol
0,1,B,Sim,Baixo,33,85.1,186,199.63
1,2,A,Não,Moderado,68,105.0,184,236.98
2,3,O,Não,Alto,25,64.8,180,161.79
3,4,A,Não,Alto,43,120.2,167,336.24
4,5,AB,Não,Baixo,79,88.5,175,226.23
5,6,B,Não,Baixo,68,66.8,170,185.31
6,7,A,Sim,Baixo,60,117.3,181,289.33
7,8,O,Sim,Moderado,35,86.9,174,216.48
8,9,O,Não,Baixo,62,81.3,166,235.30
9,10,B,Sim,Alto,44,32.7,165,97.79


In [5]:
# Ajustar dataframe
df.drop('Id', axis=1, inplace=True)

#aplicar one hot nas variaveis categoricas
df = pd.get_dummies(df, columns=['Grupo Sanguíneo', 'Fumante', 'Nível de Atividade'])

In [6]:
df

,Idade,Peso,Altura,Colesterol,Grupo Sanguíneo_A,Grupo Sanguíneo_AB,Grupo Sanguíneo_B,Grupo Sanguíneo_O,Fumante_Não,Fumante_Sim,Nível de Atividade_Alto,Nível de Atividade_Baixo,Nível de Atividade_Moderado
0,33,85.1,186,199.63,False,False,True,False,False,True,False,True,False
1,68,105.0,184,236.98,True,False,False,False,True,False,False,False,True
2,25,64.8,180,161.79,False,False,False,True,True,False,True,False,False
3,43,120.2,167,336.24,True,False,False,False,True,False,True,False,False
4,79,88.5,175,226.23,False,True,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,31,68.1,166,206.81,False,True,False,False,False,True,False,False,True
996,51,47.7,170,128.03,False,False,False,True,True,False,True,False,False
997,39,85.5,176,211.14,False,True,False,False,True,False,False,True,False
998,61,91.2,161,284.53,False,True,False,False,False,True,False,True,False


### Treinar modelo - Regressão

In [7]:
#Separar x e y
X = df.drop('Colesterol', axis=1)
y = df['Colesterol']

In [8]:
#Separar Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=51)

In [9]:
#Treinar modelo com RFE
# Recursive feature eminination
# uso um estimador e defino uma quantidade de features (dois hiperparametros)
#RFE faz iterações niciando com todas as features e eliminando a cada iteração até atingir a quantidade definida
# Elimina as caracteristicas menos importantes
rfe_method = RFE(estimator=LinearRegression(), n_features_to_select=2)
rfe_method.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [10]:
#Quais features foram selecionadas
X_train.columns[(rfe_method.get_support())]

Index(['Peso', 'Altura'], dtype='object')

In [11]:
#Ranking de features
def mostrar_ranking(metodo_fs, X_train):

    #obter ranking de features
    ranking = rfe_method.ranking_

    #Obter nomes
    nomes_features = X_train.columns.to_list()

    #Crie um dataframe com os rankings e os nomes da =s features
    df_ranking = pd.DataFrame({'Feature': nomes_features, 'Ranking': ranking})

    #ordene o dataFrame pelo ranking
    df_ranking = df_ranking.sort_values(by='Ranking')

    #Exibir ranking
    print(df_ranking)

In [12]:
# Ranking features do RFE
mostrar_ranking(rfe_method, X_train)

                        Feature  Ranking
1                          Peso        1
2                        Altura        1
9       Nível de Atividade_Alto        2
7                   Fumante_Não        3
4            Grupo Sanguíneo_AB        4
8                   Fumante_Sim        5
5             Grupo Sanguíneo_B        6
3             Grupo Sanguíneo_A        7
11  Nível de Atividade_Moderado        8
10     Nível de Atividade_Baixo        9
6             Grupo Sanguíneo_O       10
0                         Idade       11


In [13]:
#Fução para avaliar performace

def performace_regressao(modelo, X_test, y_test):
    #faz a predição com modelo no conjunto de testes
    y_pred = modelo.predict(X_test)
    #Avaliar desempenho
    rmse = mean_squared_error(y_test, y_pred)
    mean_squared = np.sqrt(rmse)
    return mean_squared



In [14]:
#Performace regressão com RFE
performace_regressao(rfe_method, X_test, y_test)

np.float64(9.075343446947082)

### Treinar modelo sem rfe

In [15]:
# Treinar modelo de regressao sem rfe
model_reg = LinearRegression()
model_reg.fit(X_train, y_train)

LinearRegression()

In [16]:
#performace regressao sem rfe
performace_regressao(model_reg, X_test, y_test)

np.float64(9.017674671581466)

### Treinar modelo de regressao com RFECV

In [17]:
rfe_method_cv = RFECV(estimator=LinearRegression(), min_features_to_select=6, cv=5)
rfe_method_cv.fit(X_train, y_train)

RFECV(cv=5, estimator=LinearRegression(), min_features_to_select=6)

In [18]:
#QUal foi a performace da regressao
performace_regressao(rfe_method_cv, X_test, y_test)

np.float64(9.022882331203347)

In [19]:
#features escolhidas
X_train.columns[(rfe_method_cv.get_support())]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_A', 'Grupo Sanguíneo_AB',
       'Grupo Sanguíneo_B', 'Grupo Sanguíneo_O', 'Fumante_Não', 'Fumante_Sim',
       'Nível de Atividade_Alto', 'Nível de Atividade_Baixo',
       'Nível de Atividade_Moderado'],
      dtype='object')

In [20]:
#Quantas features foram selecionadas
rfe_method_cv.n_features_

np.int64(11)

### Treinar modelo de regressao com selectfrommodel

In [21]:

sfm_method = SelectFromModel(estimator=model_reg, max_features=4, threshold=0.5)
sfm_method.fit(X_train, y_train)

SelectFromModel(estimator=LinearRegression(), max_features=4, threshold=0.5)

In [22]:
#features escolhidas
X_train.columns[(sfm_method.get_support())]

Index(['Peso', 'Altura', 'Grupo Sanguíneo_AB', 'Nível de Atividade_Alto'], dtype='object')

In [23]:
# treinar modelo com as features selecionadas
X_train_ajustado_reg = sfm_method.transform(X_train)
X_test_ajustado_reg = sfm_method.transform(X_test)
model_reg.fit(X_train_ajustado_reg, y_train)

LinearRegression()

In [24]:
performace_regressao(model_reg, X_test_ajustado_reg, y_test)

np.float64(9.024674426990273)